In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import os
import matplotlib.image as mpimg
import pandas as pd
from keras.utils import plot_model
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [2]:
%matplotlib inline

## Model

In [3]:
def iresnet_ver1(shape=(None,None,3)):
    
    activation_relu = 'relu'
    init_random_normal = 'random_normal'
    
    # Input Layer
    
    input_left = Input(shape)
    input_right = Input(shape)
    
    #Stem Block for multiscale shared feature extraction begins
    
    conv1 = Conv2D(64, 7, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv1')
    conv1_left = conv1(input_left)
    conv1_right = conv1(input_right)
    
    up1 = Conv2DTranspose(32, 4, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='up1')
    up1_left = up1(conv1_left)
    up1_right = up1(conv1_right)
    
    conv2 = Conv2D(128, 5, strides=2, activation=activation_relu, kernel_initializer=init_random_normal, name='conv2')
    conv2_left = conv2(conv1_left)
    conv2_right = conv2(conv1_right)
    
    up2 = Conv2DTranspose(32, 8, strides=4, activation=activation_relu, kernel_initializer=init_random_normal, name='up2')
    up2_left = up2(conv2_left)
    up2_right = up2(conv2_right)
    
    match_conv1 = Conv2D(32, 5, strides=1, activation=activation_relu,kernel_initializer=init_random_normal, name='matchconv1')
    match_conv1_left = match_conv1(up1_left)
    match_conv1_right = match_conv1(up1_right)
    
    up1_2_left = concatenate([match_conv1_left,up2_left],name='concat_left')
    up1_2_right = concatenate([match_conv1_right,up2_right],name='conact_right')
    
    conv1_2 = Conv2D(32, 1, strides=1, activation=activation_relu, kernel_initializer=init_random_normal, name='convup1_2')
    conv1_2_left = conv1_2(up1_2_left)
    conv1_2_right = conv1_2(up1_2_right)
    
    #Stem Block for multiscale shared feature extraction ends
    
    model_shared_features = Model(inputs=[input_left,input_right],outputs=[conv1_2_left,conv1_2_right])
    model_shared_features.compile(optimizer = Adam(lr = 0.00001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model_shared_features.summary()
    plot_model(model=model_shared_features,show_layer_names=True, show_shapes=True, to_file='model_shared_features.png')
    
    #Initial Disparity Estimation Sub-network begins

    
    return model_shared_features

In [4]:
x = iresnet_ver1((640,480,3))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 640, 480, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 640, 480, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 317, 237, 64) 9472        input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
up1 (Conv2DTranspose)           (None, 636, 476, 32) 32800       conv1[0][0]                      
          